In [ ]:
pip install --quiet wikipedia sentence-transformers faiss-cpu transformers torch langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


**Reasoning**:
The required libraries have been installed. Now, I will import the necessary classes and load the MiniLM embedding model and FLAN-T5-small tokenizer and model as per the instructions.



In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load MiniLM embedding model
minilm_model = SentenceTransformer(minilm_model_name)
print(f"MiniLM embedding model '{minilm_model_name}' loaded successfully.")

# Load FLAN-T5-small tokenizer and model
flan_tokenizer = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name)
print(f"FLAN-T5-small model '{flan_model_name}' and tokenizer loaded successfully.")

MiniLM embedding model 'all-MiniLM-L6-v2' loaded successfully.
FLAN-T5-small model 'google/flan-t5-small' and tokenizer loaded successfully.


In [ ]:
import wikipedia

def fetch_wikipedia_articles(query, num_results=3):
    """
    Fetches relevant articles from Wikipedia based on a given query.

    Args:
        query (str): The search query for Wikipedia articles.
        num_results (int): The maximum number of search results to process.

    Returns:
        list: A list of dictionaries, where each dictionary contains 'title', 'url',
              and 'content' for a fetched Wikipedia article.
    """
    articles = []
    try:
        # Search for potential Wikipedia page titles
        search_titles = wikipedia.search(query, results=num_results)
        print(f"Found {len(search_titles)} potential Wikipedia pages for '{query}': {search_titles}")

        for title in search_titles:
            try:
                # Retrieve the Wikipedia page object
                page = wikipedia.page(title, auto_suggest=False, redirect=True)
                articles.append({
                    'title': page.title,
                    'url': page.url,
                    'content': page.content
                })
            except wikipedia.exceptions.PageError:
                print(f"Page '{title}' not found. Skipping.")
            except wikipedia.exceptions.DisambiguationError as e:
                print(f"Disambiguation error for '{title}'. Options: {e.options}. Skipping.")
            except Exception as e:
                print(f"An unexpected error occurred while fetching '{title}': {e}. Skipping.")

    except Exception as e:
        print(f"An error occurred during Wikipedia search for '{query}': {e}")

    return articles

# Test the function with the provided query variable
fetched_wikipedia_articles = fetch_wikipedia_articles(query)

print("\nFetched articles:")
if fetched_wikipedia_articles:
    for article in fetched_wikipedia_articles:
        print(f"Title: {article['title']}")
        print(f"URL: {article['url']}")
        print("\n")
else:
    print("No articles fetched.")

Found 3 potential Wikipedia pages for 'Retrieval Augmented Generation': ['Retrieval-augmented generation', 'Vector database', 'Knowledge cutoff']

Fetched articles:
Title: Retrieval-augmented generation
URL: https://en.wikipedia.org/wiki/Retrieval-augmented_generation


Title: Vector database
URL: https://en.wikipedia.org/wiki/Vector_database


Title: Knowledge cutoff
URL: https://en.wikipedia.org/wiki/Knowledge_cutoff




In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instantiate RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

# Create an empty list to store all chunks
all_chunks = []

# Iterate through each article and split its content into chunks
for article in fetched_wikipedia_articles:
    doc_chunks = text_splitter.split_text(article['content'])
    for chunk_text in doc_chunks:
        all_chunks.append({
            'text': chunk_text,
            'title': article['title'],
            'url': article['url']
        })

# Print the total number of chunks created
print(f"Total number of chunks created: {len(all_chunks)}")

# Print the first few chunks to verify the splitting process
print("\nFirst 3 chunks:")
for i, chunk in enumerate(all_chunks[:3]):
    print(f"Chunk {i+1}:")
    print(f"  Title: {chunk['title']}")
    print(f"  URL: {chunk['url']}")
    print(f"  Content: {chunk['text'][:200]}...") # Display first 200 chars
    print("---\n")

Total number of chunks created: 67

First 3 chunks:
Chunk 1:
  Title: Retrieval-augmented generation
  URL: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
  Content: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information from external data sources. With RAG, LLMs do not respond to u...
---

Chunk 2:
  Title: Retrieval-augmented generation
  URL: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
  Content: data. For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources....
---

Chunk 3:
  Title: Retrieval-augmented generation
  URL: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
  Content: RAG improves large language models (LLMs) by incorporating information retrieval before generating responses. Unlike LLMs that rely on static training data, RAG pulls relevant text from databases, upl...
---



In [ ]:
import numpy as np

# 1. Extract the 'text' content from each dictionary in the all_chunks list
chunk_texts = [chunk['text'] for chunk in all_chunks]

# 2. Use the loaded minilm_model to generate embeddings for all the chunk_texts
chunk_embeddings = minilm_model.encode(chunk_texts, show_progress_bar=True)

# 3. Store the generated embeddings in a variable (already done above as chunk_embeddings)

# 4. Print the shape of the chunk_embeddings
print(f"Shape of generated embeddings: {chunk_embeddings.shape}")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Shape of generated embeddings: (67, 384)


In [ ]:
import faiss

# 1. Get the dimensionality of the embeddings
dimension = chunk_embeddings.shape[1]

# 2. Initialize a FAISS IndexFlatL2 index
# IndexFlatL2 calculates Euclidean distance (L2 norm)
faiss_index = faiss.IndexFlatL2(dimension)

# 3. Add the chunk_embeddings to the FAISS index
# FAISS requires the input to be float32
faiss_index.add(np.array(chunk_embeddings).astype('float32'))

# 4. Print the number of vectors in the index to confirm it has been populated correctly
print(f"Number of vectors in the FAISS index: {faiss_index.ntotal}")

Number of vectors in the FAISS index: 67


In [ ]:
def retrieve_chunks(query, k=3):
    """
    Retrieves the top-k most relevant text chunks for a given query using the FAISS index.

    Args:
        query (str): The user's query string.
        k (int): The number of top relevant chunks to retrieve.

    Returns:
        list: A list of dictionaries, where each dictionary represents a relevant text chunk
              including its text, title, and URL.
    """
    # 1. Convert the query into an embedding using the MiniLM model
    query_embedding = minilm_model.encode([query])

    # 2. Ensure the query embedding is float32 and 2D array for FAISS
    query_embedding = np.array(query_embedding).astype('float32')

    # 3. Utilize faiss_index.search() to find the k most similar embeddings
    # D: distances, I: indices of the nearest neighbors
    distances, indices = faiss_index.search(query_embedding, k)

    # 4. Access and compile the corresponding text chunks from the all_chunks list
    relevant_chunks = []
    for i in indices[0]: # indices[0] because query_embedding is a 2D array (batch of 1)
        relevant_chunks.append(all_chunks[i])

    return relevant_chunks

# Test the function with an example query
example_query = query # Using the predefined 'query' variable
retrieved_k = 3

print(f"\nRetrieving top {retrieved_k} chunks for query: '{example_query}'")
retrieved_chunks = retrieve_chunks(example_query, k=retrieved_k)

if retrieved_chunks:
    print(f"Found {len(retrieved_chunks)} relevant chunks:")
    for i, chunk in enumerate(retrieved_chunks):
        print(f"Chunk {i+1} (Title: {chunk['title']}):")
        print(f"  Content: {chunk['text'][:300]}...") # Display first 300 chars
        print(f"  URL: {chunk['url']}")
        print("---")
else:
    print("No relevant chunks found.")


Retrieving top 3 chunks for query: 'Retrieval Augmented Generation'
Found 3 relevant chunks:
Chunk 1 (Title: Retrieval-augmented generation):
  Content: Retrieval-augmented generation (RAG) enhances large language models (LLMs) by incorporating an information-retrieval mechanism that allows models to access and utilize additional data beyond their original training set. Ars Technica notes that "when new information becomes available, rather than hav...
  URL: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
---
Chunk 2 (Title: Knowledge cutoff):
  Content: == Attempts to overcome knowledge cutoffs ==


=== Retrieval-augmented generation ===...
  URL: https://en.wikipedia.org/wiki/Knowledge_cutoff
---
Chunk 3 (Title: Vector database):
  Content: recommendations engines, object detection, and retrieval-augmented generation (RAG)....
  URL: https://en.wikipedia.org/wiki/Vector_database
---


In [ ]:
import torch

def generate_answer(query, retrieved_chunks):
    """
    Generates a grounded answer to the user's query using the FLAN-T5-small model
    and the provided retrieved chunks as context.

    Args:
        query (str): The user's query string.
        retrieved_chunks (list): A list of dictionaries, where each dictionary
                                 represents a relevant text chunk.

    Returns:
        str: The generated answer string.
    """
    # 1. Combine retrieved chunks to form the context
    context = " ".join([chunk['text'] for chunk in retrieved_chunks])

    # 2. Construct the prompt for the FLAN-T5 model
    # Format: 'Context: [chunk1_text] [chunk2_text] ... Question: [user_query]'
    prompt = f"Context: {context} Question: {query}"

    # 3. Use the flan_tokenizer to tokenize the constructed prompt
    inputs = flan_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # 4. Pass the tokenized input to the flan_model to generate an answer
    # Using appropriate generation parameters
    outputs = flan_model.generate(
        inputs["input_ids"],
        max_new_tokens=150, # Maximum number of tokens to generate
        do_sample=True,     # Use sampling
        top_k=50,           # Consider top 50 tokens for sampling
        top_p=0.95,         # Nucleus sampling
        temperature=0.7,    # Sampling temperature
        num_return_sequences=1 # Only generate one sequence
    )

    # 5. Decode the generated token IDs back into a human-readable string
    answer = flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

# Test the generate_answer function
print(f"\nGenerating answer for query: '{example_query}'")
final_answer = generate_answer(example_query, retrieved_chunks)

print("\nGenerated Answer:")
print(final_answer)



Generating answer for query: 'Retrieval Augmented Generation'

Generated Answer:
explains the main purpose of augmentations
